In [1]:
import cv2
import os
import numpy as np
import time
import os.path as osp
import tensorflow as tf

In [2]:
print(os.getcwd())

C:\Users\Manivas\Downloads\CenterNet


In [3]:
from generators.pascal import PascalVocGenerator
from generators.csv_ import CSVGenerator
from models.resnet import centernet
from generators.utils import affine_transform, get_affine_transform
from eval.common import _get_detections, _get_annotations, evaluate, _compute_ap

In [4]:
args_test_annotations_path = 'Ouhands_bounding/test_colour/test_annot.csv'
args_classes_path = 'Ouhands_bounding/classes.csv'

In [5]:
generator = CSVGenerator(
    args_test_annotations_path,
    args_classes_path,
    shuffle_groups=True,
)

generator_input_size = 512
generator_num_classes = 10
generator_classes = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4,
    'F': 5,
    'H': 6,
    'I': 7,
    'J': 8,
    'K': 9
}

In [6]:
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
'''generator = PascalVocGenerator(
    'pascal_datasets/test/VOC2007',
    'test',
    shuffle_groups=False,
    skip_truncated=False,
    skip_difficult=True,
)

generator_input_size = 512
generator_num_classes = 20
generator_classes = {
    'aeroplane': 0,
    'bicycle': 1,
    'bird': 2,
    'boat': 3,
    'bottle': 4,
    'bus': 5,
    'car': 6,
    'cat': 7,
    'chair': 8,
    'cow': 9,
    'diningtable': 10,
    'dog': 11,
    'horse': 12,
    'motorbike': 13,
    'person': 14,
    'pottedplant': 15,
    'sheep': 16,
    'sofa': 17,
    'train': 18,
    'tvmonitor': 19
}'''

"generator = PascalVocGenerator(\n    'pascal_datasets/test/VOC2007',\n    'test',\n    shuffle_groups=False,\n    skip_truncated=False,\n    skip_difficult=True,\n)\n\ngenerator_input_size = 512\ngenerator_num_classes = 20\ngenerator_classes = {\n    'aeroplane': 0,\n    'bicycle': 1,\n    'bird': 2,\n    'boat': 3,\n    'bottle': 4,\n    'bus': 5,\n    'car': 6,\n    'cat': 7,\n    'chair': 8,\n    'cow': 9,\n    'diningtable': 10,\n    'dog': 11,\n    'horse': 12,\n    'motorbike': 13,\n    'person': 14,\n    'pottedplant': 15,\n    'sheep': 16,\n    'sofa': 17,\n    'train': 18,\n    'tvmonitor': 19\n}"

In [7]:
'''def preprocess_image(image, c, s, tgt_w, tgt_h):
  trans_input = get_affine_transform(c, s, (tgt_w, tgt_h))
  image = cv2.warpAffine(image, trans_input, (tgt_w, tgt_h), flags=cv2.INTER_LINEAR)
  image = image.astype(np.float32)

  image[..., 0] -= 103.939
  image[..., 1] -= 116.779
  image[..., 2] -= 123.68

  image[..., 0] /= 255.0
  image[..., 1] /= 255.0
  image[..., 2] /= 255.0

  return image'''

'def preprocess_image(image, c, s, tgt_w, tgt_h):\n  trans_input = get_affine_transform(c, s, (tgt_w, tgt_h))\n  image = cv2.warpAffine(image, trans_input, (tgt_w, tgt_h), flags=cv2.INTER_LINEAR)\n  image = image.astype(np.float32)\n\n  image[..., 0] -= 103.939\n  image[..., 1] -= 116.779\n  image[..., 2] -= 123.68\n\n  image[..., 0] /= 255.0\n  image[..., 1] /= 255.0\n  image[..., 2] /= 255.0\n\n  return image'

In [8]:
model_path = 'checkpoints/End2End_Resnet50_danet_1_normalization_2_2000/csv_57_0.8407_1.1972_2.2595.h5'
num_classes = generator_num_classes
classes = list(generator_classes.keys())
flip_test = False
nms = True
keep_resolution = False
score_threshold = 0.2
iou_threshold = 0.5
max_detections = 100
colors = [np.random.randint(0, 256, 3).tolist() for i in range(num_classes)]
model, prediction_model, debug_model = centernet(num_classes=num_classes,
                                                 backbone='resnet50',
                                                 score_threshold=score_threshold,
                                                 nms=nms,
                                                 flip_test=flip_test
                                                 )
prediction_model.load_weights(model_path, by_name=True)

In [9]:
'''# run evaluation
average_precisions, f1_scores = evaluate(
            generator,
            prediction_model,
            iou_threshold=iou_threshold,
            score_threshold=score_threshold,
            max_detections=max_detections,
            visualize=False
        )


# compute per class average precision
total_instances = []
precisions = []
for label, (average_precision, num_annotations) in average_precisions.items():
    print('{:.0f} instances of class'.format(num_annotations),
          generator.label_to_name(label), 'with average precision: {:.4f}'.format(average_precision))
    total_instances.append(num_annotations)
    precisions.append(average_precision)

mean_ap = sum(precisions) / sum(x > 0 for x in total_instances)
print('mAP: {:.4f}'.format(mean_ap))


# compute per class F1-score
total_instances = []
scores = []
for label, f1_score in f1_scores.items():
    print('{:.0f} instances of class'.format(num_annotations),
              generator.label_to_name(label), 'with F1-score: {:.4f}'.format(f1_score))
    total_instances.append(num_annotations)
    scores.append(f1_score)

mean_f1_score = sum(scores) / sum(x > 0 for x in total_instances)
print('Mean F1-score: {:.4f}'.format(mean_f1_score))'''

"# run evaluation\naverage_precisions, f1_scores = evaluate(\n            generator,\n            prediction_model,\n            iou_threshold=iou_threshold,\n            score_threshold=score_threshold,\n            max_detections=max_detections,\n            visualize=False\n        )\n\n\n# compute per class average precision\ntotal_instances = []\nprecisions = []\nfor label, (average_precision, num_annotations) in average_precisions.items():\n    print('{:.0f} instances of class'.format(num_annotations),\n          generator.label_to_name(label), 'with average precision: {:.4f}'.format(average_precision))\n    total_instances.append(num_annotations)\n    precisions.append(average_precision)\n\nmean_ap = sum(precisions) / sum(x > 0 for x in total_instances)\nprint('mAP: {:.4f}'.format(mean_ap))\n\n\n# compute per class F1-score\ntotal_instances = []\nscores = []\nfor label, f1_score in f1_scores.items():\n    print('{:.0f} instances of class'.format(num_annotations),\n              

In [10]:
for i in range(300,325):
    image = generator.load_image(i)
    #path = 'pascal_datasets/test/VOC2007/JPEGImages/009915.jpg'
    #image = cv2.imread(path)
    src_image = image.copy()

    c = np.array([image.shape[1] / 2., image.shape[0] / 2.], dtype=np.float32)
    s = max(image.shape[0], image.shape[1]) * 1.0

    tgt_w = generator.input_size
    tgt_h = generator.input_size
    image = generator.preprocess_image(image, c, s, tgt_w=tgt_w, tgt_h=tgt_h)
    if flip_test:
        flipped_image = image[:, ::-1]
        inputs = np.stack([image, flipped_image], axis=0)
    else:
        inputs = np.expand_dims(image, axis=0)

    # run network
    start = time.time()
    detections = prediction_model.predict_on_batch(inputs)[0]
    print("Elapsed time: ", time.time() - start)
    scores = detections[:, 4]

    # sort by score
    indices = np.argsort(-scores)[0]

    if (scores[indices]>=score_threshold):
        # select those detections
        detections = np.expand_dims(detections[indices], axis=0)

        detections_copy = detections.copy()
        detections = detections.astype(np.float64)
        trans = get_affine_transform(c, s, (tgt_w // 4, tgt_h // 4), inv=1)

        for j in range(detections.shape[0]):
            detections[j, 0:2] = affine_transform(detections[j, 0:2], trans)
            detections[j, 2:4] = affine_transform(detections[j, 2:4], trans)

            detections[:, [0, 2]] = np.clip(detections[:, [0, 2]], 0, src_image.shape[1])
            detections[:, [1, 3]] = np.clip(detections[:, [1, 3]], 0, src_image.shape[0])

        '''all_detections = []
        for class_id in range(generator.num_classes()):
        class_detections = tf.boolean_mask(detections, tf.equal(detections[:, 5], class_id))
        #print("For class-{} Class detections shape before:".format(class_id), class_detections.shape)
        nms_keep_indices = tf.image.non_max_suppression(tf.cast(class_detections[:, :4],tf.float32),
                                                        tf.cast(class_detections[:, 4],tf.float32),
                                                        5,
                                                        iou_threshold=iou_threshold,
                                                        score_threshold=score_threshold)
        class_detections = tf.gather(class_detections, nms_keep_indices)

        if (class_detections.shape[0]>0):
          all_detections.append(class_detections)

        if (all_detections!=[]):  
        all_detections = tf.concat(all_detections, axis=0)'''

        for detection in detections:
            #detection = detections.numpy()
            xmin = int(round(detection[0]))
            ymin = int(round(detection[1]))
            xmax = int(round(detection[2]))
            ymax = int(round(detection[3]))
            score = '{:.4f}'.format(detection[4])
            class_id = int(detection[5])
            color = colors[class_id]
            class_name = classes[class_id]
            label = '-'.join([class_name, score])
            ret, baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 1, 3)
            cv2.rectangle(src_image, (xmin, ymin), (xmax, ymax), color, 3)
            cv2.rectangle(src_image, (xmin, ymax - ret[1] - baseline), (xmin + ret[0], ymax), color, -1)
            cv2.putText(src_image, label, (xmin, ymax - baseline), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 3)

    #image_fname = 'test_{}'.format(i)
    #cv2.imwrite('test/{}.jpg'.format(image_fname), src_image)

Elapsed time:  2.8663246631622314
Elapsed time:  0.44492077827453613
Elapsed time:  0.4837040901184082
Elapsed time:  0.4653587341308594
Elapsed time:  0.4469616413116455
Elapsed time:  0.434661865234375
Elapsed time:  0.4513859748840332
Elapsed time:  0.44323158264160156
Elapsed time:  0.4564692974090576
Elapsed time:  0.4426560401916504
Elapsed time:  0.42006897926330566
Elapsed time:  0.4316840171813965
Elapsed time:  0.42943358421325684
Elapsed time:  0.44704627990722656
Elapsed time:  0.42949342727661133
Elapsed time:  0.44109296798706055
Elapsed time:  0.46184539794921875
Elapsed time:  0.45400404930114746
Elapsed time:  0.46065306663513184
Elapsed time:  0.4780080318450928
Elapsed time:  0.45837926864624023
Elapsed time:  0.44489169120788574
Elapsed time:  0.43922972679138184
Elapsed time:  0.45816802978515625
Elapsed time:  0.4402801990509033


In [11]:
# Object to capture the live video using the default camera of PC
cap = cv2.VideoCapture(0)


# Reading each frame of the live video and making detections on the video frames

while True:
    
    
    
    # Reading the frame of captured vide
    _,image = cap.read()


    src_image = image.copy()

    c = np.array([image.shape[1] / 2., image.shape[0] / 2.], dtype=np.float32)
    s = max(image.shape[0], image.shape[1]) * 1.0

    tgt_w = generator.input_size
    tgt_h = generator.input_size
    image = generator.preprocess_image(image, c, s, tgt_w=tgt_w, tgt_h=tgt_h)
    if flip_test:
        flipped_image = image[:, ::-1]
        inputs = np.stack([image, flipped_image], axis=0)
    else:
        inputs = np.expand_dims(image, axis=0)

        
    # Start time
    start_time = time.time()
    
    # run network
    detections = prediction_model.predict_on_batch(inputs)[0]
    
    scores = detections[:, 4]

    # sort by score
    indices = np.argsort(-scores)[0]
    
    # End time
    end_time = time.time()
    
    elapsed_time = end_time-start_time
    
    print("Elapsed time: ", elapsed_time)

    if (scores[indices]>=score_threshold):
        # select those detections
        detections = np.expand_dims(detections[indices], axis=0)

        detections_copy = detections.copy()
        detections = detections.astype(np.float64)
        trans = get_affine_transform(c, s, (tgt_w // 4, tgt_h // 4), inv=1)

        for j in range(detections.shape[0]):
            detections[j, 0:2] = affine_transform(detections[j, 0:2], trans)
            detections[j, 2:4] = affine_transform(detections[j, 2:4], trans)

            detections[:, [0, 2]] = np.clip(detections[:, [0, 2]], 0, src_image.shape[1])
            detections[:, [1, 3]] = np.clip(detections[:, [1, 3]], 0, src_image.shape[0])

        '''all_detections = []
        for class_id in range(generator.num_classes()):
        class_detections = tf.boolean_mask(detections, tf.equal(detections[:, 5], class_id))
        #print("For class-{} Class detections shape before:".format(class_id), class_detections.shape)
        nms_keep_indices = tf.image.non_max_suppression(tf.cast(class_detections[:, :4],tf.float32),
                                                        tf.cast(class_detections[:, 4],tf.float32),
                                                        5,
                                                        iou_threshold=iou_threshold,
                                                        score_threshold=score_threshold)
        class_detections = tf.gather(class_detections, nms_keep_indices)

        if (class_detections.shape[0]>0):
          all_detections.append(class_detections)

        if (all_detections!=[]):  
        all_detections = tf.concat(all_detections, axis=0)'''

        for detection in detections:
            #detection = detections.numpy()
            xmin = int(round(detection[0]))
            ymin = int(round(detection[1]))
            xmax = int(round(detection[2]))
            ymax = int(round(detection[3]))
            score = '{:.4f}'.format(detection[4])
            class_id = int(detection[5])
            color = colors[class_id]
            class_name = classes[class_id]
            label = '-'.join([class_name, score])
            ret, baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 1, 3)
            cv2.rectangle(src_image, (xmin, ymin), (xmax, ymax), color, 3)
            cv2.rectangle(src_image, (xmin, ymax - ret[1] - baseline), (xmin + ret[0], ymax), color, -1)
            cv2.putText(src_image, label, (xmin, ymax - baseline), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 3)

    
    fps = 1/elapsed_time
    
    #cv2.putText(src_image, "FPS: " + str(round(elapsed_time, 4)), (300, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)
    
    # Display the marked images with a duration of 1 ms 
    # Click on the camera window created by cv2
    cv2.imshow("Objects detection on live video", src_image)
        
    # Press key q to stop the live detection and turn off the camera
    if ord("q") == cv2.waitKey(1):
        break

# Releases the I/O device (i.e.,camera) by opencv            
cap.release()

# Close all the opened windows
cv2.destroyAllWindows()

Elapsed time:  0.4792916774749756
Elapsed time:  0.41965770721435547
Elapsed time:  0.4160313606262207
Elapsed time:  0.39447546005249023
Elapsed time:  0.5317249298095703
Elapsed time:  0.4207019805908203
Elapsed time:  0.38851118087768555
Elapsed time:  0.39091014862060547
Elapsed time:  0.3981137275695801
Elapsed time:  0.3961944580078125
Elapsed time:  0.3963735103607178
Elapsed time:  0.39096879959106445
Elapsed time:  0.3924553394317627
Elapsed time:  0.39340901374816895
Elapsed time:  0.3876042366027832
Elapsed time:  0.39246702194213867
Elapsed time:  0.3862922191619873
Elapsed time:  0.3928217887878418
Elapsed time:  0.40975046157836914
Elapsed time:  0.3905797004699707
Elapsed time:  0.39238953590393066
Elapsed time:  0.3893280029296875
Elapsed time:  0.4047279357910156
Elapsed time:  0.3883330821990967
Elapsed time:  0.40960693359375
Elapsed time:  0.4321587085723877
Elapsed time:  0.4664921760559082
Elapsed time:  0.4569876194000244
Elapsed time:  0.47500038146972656
Elapse